This notebook contains all of our ideas for what should be included in the EDA.

In [2]:
#Library Imports --- IGNORE
import numpy as np
import pandas as pd
import matplotlib as plt
import os

from pathlib import Path

In [ ]:
#Jun - Reading the files
df_exreviews = pd.read_excel(os.path.join("..", "Metacritic dataset", "ExpertReviews.xlsx"))
df_metaclean = pd.read_excel(os.path.join("..", "Metacritic dataset", "metaClean43Brightspace.xlsx"))
df_sales = pd.read_excel(os.path.join("..", "Metacritic dataset", "sales.xlsx"))
df_usreviews = pd.read_excel(os.path.join("..", "Metacritic dataset", "UserReviews.xlsx"))


In [ ]:
# Rosie - Lowercase, trim, remove special characters
# df1


def clean_text_column(text):
    if isinstance(text, str):
        text = text.lower()                      # lowercase
        text = text.strip()                      # remove spaces at start/end
        text = re.sub(r'[^a-z0-9\s]', '', text)  # remove special characters
    return text
for col in df1.columns:
    df1[col] = df1[col].apply(clean_text_column)
for col in df2.columns:
    df2[col] = df2[col].apply(clean_text_column)  
for col in df3.columns:
    df3[col] = df3[col].apply(clean_text_column)  
for col in df4.columns:
    df4[col] = df4[col].apply(clean_text_column)



In [ ]:
#Rosie - number of rows and columns in df3
print("There are {} rows and {} columns in UserReviews".format(df3.shape[0], df3.shape[1]))
print("There are {} rows and {} columns in ExpertReviews".format(df4.shape[0], df4.shape[1]))

In [ ]:
# Rosie - a quick overview of the dataset by looking at the first and last few rows df3
df3.head()
df3.tail()

In [ ]:
# Rosie - a quick overview of the dataset by looking at the first and last few rows df4
df4.head()
df4.tail()

In [ ]:
# Rosie - check the unique df3
print("Is URL column unique? :", df3['url'].is_unique)
print("Number of duplicate URLs:", df3['url'].duplicated().sum())
print("Number of unique URLs   :", df3['url'].nunique())
print("Total number of rows    :", df3.shape[0])

print("Is URL column unique? :", df3['reviewer'].is_unique)
print("Number of duplicate URLs:", df3['reviewer'].duplicated().sum())
print("Number of unique URLs   :", df3['reviewer'].nunique())
print("Total number of rows    :", df3.shape[0])


In [ ]:
# Rosie - check the unique df4
print("Is URL column unique? :", df4['url'].is_unique)
print("Number of duplicate URLs:", df4['url'].duplicated().sum())
print("Number of unique URLs   :", df4['url'].nunique())
print("Total number of rows    :", df4.shape[0])

print("Is URL column unique? :", df4['reviewer'].is_unique)
print("Number of duplicate URLs:", df4['reviewer'].duplicated().sum())
print("Number of unique URLs   :", df4['reviewer'].nunique())
print("Total number of rows    :", df4.shape[0])

In [ ]:
# Rosie - missing values df3
n_nulls_df3 = df3.loc[lambda x: x.isnull().any(axis=1)].shape
print(f"There are {n_nulls_df3[0]} rows with missing values in df3")
df3.isna().sum()
# 1. Drop rows missing review text
df3 = df3.dropna(subset=['Rev']).copy()

# 2. Fill missing reviewer names
df3.loc[:, 'reviewer'] = df3['reviewer'].fillna('unknown_reviewer')

# 3. Fill missing score only if needed
# df3.loc[:, 'idvscore'] = df3['idvscore'].fillna(-1)  # or drop them if needed

# 4. Replace missing thumbsUp and thumbsTot with 0
df3.loc[:, 'thumbsUp'] = df3['thumbsUp'].fillna(0)
df3.loc[:, 'thumbsTot'] = df3['thumbsTot'].fillna(0)

In [ ]:
#  Rosie - Check missing values 
n_nulls_df4 = df4.loc[lambda x: x.isnull().any(axis=1)].shape
print(f"There are {n_nulls_df4[0]} rows with missing values in df4")

print("\nMissing values per column before cleaning:")
print(df4.isna().sum())

#  Drop rows where review text is missing 
df4 = df4.dropna(subset=['Rev']).copy()

#  Fill optional missing values 

df4.loc[:, 'reviewer'] = df4['reviewer'].fillna('unknown_reviewer')

# Missing date 
df4.loc[:, 'dateP'] = df4['dateP'].fillna('unknown_date')

# Check again after cleaning
print("\nMissing values per column after cleaning:")
print(df4.isna().sum())


In [ ]:
# Iro - Clean column names from sales 
sales.columns = (
    sales.columns
    .str.lower()
    .str.strip()
    .str.replace(" ", "_") 
    .str.replace(r"[^0-9a-z_]", "", regex=True)
)
sales.columns

In [ ]:
# Iro - Clean text column from sales 
text_cols = ["title", "genre", "keywords", "creative_type", "url"]

for col in text_cols:
    if col in sales.columns:
        sales[col] = (
            sales[col].astype(str)
            .str.lower()
            .str.strip()
        )

In [ ]:
# Iro - Convert numeric columns
num_cols = [
    "domestic_box_office", "international_box_office", "worldwide_box_office",
    "production_budget", "opening_weekend", "theatre_count",
    "avg_run_per_theatre", "runtime", "year"
]

for col in num_cols:
    if col in sales.columns:
        sales[col] = pd.to_numeric(sales[col], errors="coerce")

In [ ]:
# Iro - Missing value summary
sales.isna().sum() # how many missing values each column has

Data quality:
- production_budget → 26,132 missing
- domestic_box_office → 18,728 missing
- international_box_office → 9,037 missing
- unnamed_8 → 30,612 missing 

Usable columns:
- year, title, genre → 0 missing 
- runtime → 6,053 missing

In [ ]:
# Iro - Plot sales distributions
cols = ["domestic_box_office", "international_box_office", "worldwide_box_office"]
fig, axes = plt.subplots(1, 3, figsize=(12, 3))

notation = (
    "Axis meaning:\n"
    "0   →   0\n"
    "0.5 → 500M\n"
    "1.0 → 1B\n"
    "2.0 → 2B"
)

for ax, col in zip(axes, cols):
    if col in sales.columns:
        _ = ax.hist(sales[col].dropna(), bins=40)
        _ = ax.set_title(col.replace("_", " ").title())
        _ = ax.set_xlabel("Sales (USD)")
        _ = ax.set_ylabel("Frequency")

        # suppress printed return value
        _ = ax.text(
            0.98, 0.98,
            notation,
            transform=ax.transAxes,
            fontsize=8,
            ha='right',
            va='top',
            bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.7)
        )

plt.tight_layout()
plt.show()


**Interpretation of the Box Office Histograms**

The three histograms (Domestic, International, and Worldwide Box Office) show how movie revenues are distributed.
- The x-axis represents sales in USD, but very large values are shown in compressed scientific notation. The legend explains that values like 0.5, 1.0, 2.0 actually correspond to 500M, 1B, and 2B USD.
- The y-axis, labeled Frequency, indicates how many movies fall into each revenue range (bin). A higher bar means more movies in that sales interval.

Across all three charts, the very tall bar near zero shows that most movies earn very little revenue, while only a small number reach hundreds of millions or billions. This creates a long right tail, indicating that box office revenues are extremely skewed, with many low-earning films and a few very high-earning blockbusters.